Промаркируем топиками все единицы векторной базы

Инсталлы

In [1]:
! pip install --upgrade --quiet yandex-cloud-ml-sdk

Импорты

In [ ]:
!pip install yandex-cloud-ml-sdk

In [2]:
import os
from yandex_cloud_ml_sdk import YCloudML
import json
import pandas as pd
import ast


Функция для запросов к модели

In [3]:
folder_id = os.environ["FOLDER_ID"]
api_key = os.environ["API_KEY"]

In [4]:
sdk = YCloudML(folder_id=folder_id, auth=api_key)
model = sdk.models.completions("yandexgpt", model_version="rc")

In [13]:


folder_id = os.environ["FOLDER_ID"]
api_key = os.environ["API_KEY"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)
model = sdk.models.completions("yandexgpt", model_version="rc")


valid_topics = 'проходной балл, меганаправления, количество бюджетных мест, цена года обучения, общежития, индивидуальные достижения, компанни-партнеры, стажировки, математика, программирование, кафедры, преподаватели, стипендии, корпуса, ремонт, айти-этаж, лаборатории, подача заявления, дни открытых дверей, чат приемки, чаты меганаправлений, языки программирования, баллы на общежитие, сертификаты на общежитие, бви, магистратура, военная кафедра, культурные мероприятия, спортивные секции, спортивные соревнования'.split(', ') + ['мусор']
def tag_query(query):
    with open('../research/prompts/tagging_prompt.md', 'r', encoding='utf-8') as f:
        system_prompt = f.read()
    return model.run(system_prompt + "\n\n Входной запрос:\n" + query).text
tag_query('''Маник сделала, крутой капец
''')

'мусор\n'

Слияние в один датафрейм

In [4]:
result = {'source': [], 'chunk_text': []}
result['source'] += list(pd.read_csv('../data/support/merged_df.csv')['root_global_id'])
result['chunk_text'] += list(pd.read_csv('../data/support/merged_df.csv')['dialog_plain'])
result['source'] += list(pd.read_csv('../data/support/pages_mai.csv')['url'])
result['chunk_text'] += list(pd.read_csv('../data/support/pages_mai.csv')['content'])
result['source'] += list(pd.read_csv('../data/support/pdf_df.csv')['doc_name'])
result['chunk_text'] += list(pd.read_csv('../data/support/pdf_df.csv')['dialog_plain'])
result['source'] += list(pd.read_csv('../data/support/vk_chunks_df.csv')['doc_name'])
result['chunk_text'] += list(pd.read_csv('../data/support/vk_chunks_df.csv')['dialog_plain'])
result_df = pd.DataFrame(result)


In [5]:
result_df

,source,chunk_text
0,chat2021_10,возможность заселение комната знакомых друг из...
1,chat2021_10,получение временной регистрация заселиться общ...
2,chat2021_10,подать документ указать результат экзамен прий...
3,chat2021_10,целое обучение классный радовать преподаватель...
4,chat2021_10,добавить ориентироваться проходной балл предыд...
...,...,...
3876,ФРЭЛА МАИ,[ФРЭЛА МАИ 💥 В рамках VI Фестиваля радиоэлектр...
3877,ФРЭЛА МАИ,"[ФРЭЛА МАИ Уже завтра в 13:00 Василий Егоров, ..."
3878,ФРЭЛА МАИ,[ФРЭЛА МАИ МАИ приглашает студентов и школьник...
3879,ФРЭЛА МАИ,[ФРЭЛА МАИ 🚀 Попробуй свои силы — участвуй в О...


Обработка диалогов

In [ ]:
def add_query(query):
    ans = tag_query(query)

    topics = ans.strip().replace('[', '').replace(']', '').replace('\n\n\u200b', '').split(', ')
    res = []
    for i in topics:
        if i  in valid_topics:
            res.append(i)
    topics = res

    print(f'{query} -> {topics}')
    return topics

result_df['topics'] = result_df['chunk_text'].apply(add_query)

Обработка  постов

## суммаризация

In [6]:
result = {'source': [], 'dialog_plain': []}
result['source'] += list(pd.read_csv('../data/support/final_df.csv')['root_global_id'])
result['dialog_plain'] += list(pd.read_csv('../data/support/final_df.csv')['dialog_plain'])

In [8]:
result_df = pd.DataFrame(result)

In [9]:
result_df

,source,dialog_plain
0,chat2021_10,Возможность заселения в одну комнату знакомых/...
1,chat2021_10,Староста Задачи старосты\nПередавать информаци...
2,chat2021_10,Список документов для поступления (бакалавриат...
3,chat2021_10,Добрый день)\nЯляется ли обучение на направлен...
4,chat2021_10,"Доброй ночи, хотелось бы узнать как предоставл..."
...,...,...
2638,chat2023_3692,"Уважаемые абитуриенты! Если вы уверены, что ес..."
2639,chat2023_3692,"Подскажите, когда на сайте появится возможност..."
2640,chat2021_432,Модно отталкиваться от среднего балла прошлого...
2641,chat2021_12882,А сколько сейчас в итоге свободных мест в обще...


In [24]:
with open('../data/prompts/summarize_chunk.md', 'r', encoding='utf-8') as f:
    system_prompt = f.read()

In [31]:
def summarize_chunk(chunk):
    response = model.run(system_prompt + "\n\nЧанк для анализа/суммаризации `chunk_text`:\n" + chunk).text
    print(response[:100])
    return response

In [32]:
chunks = result_df['dialog_plain'].tolist()

In [33]:
summaries = []
for chunk in chunks:
    summaries.append(summarize_chunk(chunk))

result_df['sum_chunks'] = summaries

### Суть
В чанке содержится информация о возможности заселения знакомых/друзей в одну комнату в обще
### Суть

Чат содержит информацию о задачах старосты в МАИ, процессе получения временной регистрации
### Суть
В чанке перечислены списки документов, необходимых для поступления на бакалавриат, специали
### Суть
Абитуриент интересуется возможностью поступления на направления 01.03.02 и 01.03.04 на бюдж
### Суть

Обсуждение условий предоставления общежития в МАИ и возможности ознакомления с ранжированн
### Суть

В чанке обсуждаются рекомендации по выбору направлений для поступления в МАИ, а также усло
### Суть

Абитуриент интересуется процедурой поступления на направление ПМИ, включая вопросы о групп
### Суть

Студенческие клубы МАИ предлагают разнообразные возможности для творческой и спортивной де
### Суть
В МАИ есть несколько спортивных комплексов с различными секциями. Для записи в секцию нужно
### Суть

В чанке обсуждаются вопросы, связанные со стипендиями в МАИ и особенностями посту

In [34]:
result_df.to_csv('../data/support/result_df.csv')

In [ ]:
sum_chunks = result_df['sum_chunks'].tolist()